# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [1]:
!cat phdthesis.tsv

pub_date	title	venue	author	excerpt	citation	paper_url	download_url
2004-11-01	Exact and Memetic Algorithms for Two Network Design Problems	Faculty of Computer Science, Vienna University of Technology	Ivana Ljubic				publications/ljubicPhD.pdf

## Import pandas

We are using the very handy pandas library for dataframes.

In [2]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [3]:
publications = pd.read_csv("phdthesis.tsv", sep="\t", header=0)
publications.head()


,pub_date,title,venue,author,excerpt,citation,paper_url,download_url
0,2004-11-01,Exact and Memetic Algorithms for Two Network D...,"Faculty of Computer Science, Vienna University...",Ivana Ljubic,NaN,NaN,NaN,publications/ljubicPhD.pdf


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [4]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [5]:

'-'.join([i for i in 'A b'.lower().split()])

'a-b'

In [7]:
import os
for row, item in publications.iterrows():
    
#     > add url_slug creation here
#     > the download url could contain pdf location of full url
    
    url_slug = '-'.join([i for i in item.title.lower().split()])
    
    md_filename = str(item.pub_date) + "-" + url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\ntype: phdthesis"""
    
    md += "\nauthor: '" + html_escape(item.author) + "'"
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    if len(str(item.venue)) > 5:
        md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    if len(str(item.citation)) > 5:
        md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.paper_url)) > 5:
        md += "\n[View paper here](" + item.paper_url + ")\n" 
        
    if len(str(item.download_url)) > 5:
        if 'www' in str(item.download_url):
            md += "\n[Download paper here](" + item.download_url + ")\n" 
        else:
            md += "\n[Download paper here]({{site.url}}/docs/" + item.download_url + ")\n" 
        
    if len(str(item.citation)) > 5:
        md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [8]:
!ls ../_publications/

2001-01-01-solving-the-simple-plant-location-problem-by-genetic-algorithm.md
2002-01-01-evolutionary-local-search-for-the-edge-biconnectivity-augmentation-problem.md
2003-01-01-a-memetic-algorithm-for-minimum-cost-vertex-biconnectivity-augmentation-of-graphs.md
2006-01-01-an-algorithmic-framework-for-the-exact-solution-of-the-prize-collecting-steiner-tree-problem.md
2009-01-01-obtaining-optimal-k-cardinality-trees-fast.md
2010-01-01-a-branch-and-cut-and-price-algorithm-for-vertex-biconnectivity-augmentation.md
2010-01-01-orientation-based-models-for-{0,1,2}-survivable-network-design:-theory-and-practice.md
2010-01-01-the-regenerator-location-problem.md
2011-01-01-mip-models-for-connected-facility-location:-a-theoretical-and-computational-study.md
2012-01-01-exact-approaches-to-the-single-source-network-loading-problem.md
2012-01-01-qtl-mapping-using-a-memetic-algorithm-with-modifications-of-bic-as-fitness-function.md
2013-01-01-a-cutting-plane-algorithm-for-the-capacitated-connected-fa

In [17]:
!cat ../_publications/1-ilps:-application-to-fttx-network-design.md

---
title: "ILP heuristics and a new exact method for bi-objective 0/1 ILPs: Application to FTTx-network design"
collection: publications
author: 'M. Leitner, I. Ljubic, M. Sinnl, A. Werner'
permalink: /publication/2016-01-01-ilp-heuristics-and-a-new-exact-method-for-bi-objective-0/1-ilps:-application-to-fttx-network-design
date: 2016-01-01
venue: 'Comput. Oper. Res. 72: 128-146'
paperurl: 'http://www.sciencedirect.com/science/article/pii/S0305054816300302'
---
[View paper here](http://www.sciencedirect.com/science/article/pii/S0305054816300302)

[Download paper here]({{site.url}}/docs/http://homepage.univie.ac.at/markus.sinnl/wp-content/uploads/2013/09/biobjkarch-techreport.pdf)


In [ ]:
2016-01-01-ilp-heuristics-and-a-new-exact-method-for-bi-objective-ilps-application-to-fttx-network-design.md

'ilp heuristics and a new exact method for bi-objective 0/1 ilps: application to fttx-network design'

In [19]:
'-'.join([i for i in "ILP heuristics and a new exact method for bi-objective 0/1 ILPs: Application to FTTx-network design".lower().split()])

'ilp-heuristics-and-a-new-exact-method-for-bi-objective-0/1-ilps:-application-to-fttx-network-design'